<a href="https://colab.research.google.com/github/UznetDev/Data-science-home-work/blob/main/27_Sen_2024_home_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.preprocessing import *
from sklearn.feature_selection import *
from sklearn.metrics import *
from sklearn.ensemble import *
from sklearn.pipeline import *
from sklearn.compose import *
from sklearn.impute import *
from sklearn.base import *

In [3]:
df = pd.read_csv('/content/train.csv')

In [4]:
df[df['Cabin'].notna()].head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
X = df.drop(['Survived', 'PassengerId', 'Name'], axis=1)
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


numerical_cols = X.select_dtypes(include=['number']).columns
categorical_cols = X.drop(columns=numerical_cols).columns

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    # ('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=True))

])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=True))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols),
    ]
)
model = LogisticRegression()

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', model)
])
cv_accuracy = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)


print('Mean CV Accuracy:', np.mean(cv_accuracy))
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

Mean CV Accuracy: 0.8159460258051808
Accuracy: 0.8324022346368715
F1 Score: 0.7857142857142857
Precision: 0.8333333333333334
Recall: 0.7432432432432432


In [7]:
y_scor = model.predict_proba(X_test)[:, 1]

In [9]:
fpr, tpr, thresholds = roc_curve(y_test, y_scor)
roc_auc = auc(fpr, tpr)

optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

print("Best Threshold:", optimal_threshold)

Best Threshold: 0.3783464014116038


In [11]:
y_pred_optimal = (y_scor >= optimal_threshold).astype(int)

accuracy = accuracy_score(y_test, y_pred_optimal)
f1 = f1_score(y_test, y_pred_optimal)
precision = precision_score(y_test, y_pred_optimal)
recall = recall_score(y_test, y_pred_optimal)


print('Mean CV Accuracy:', np.mean(cv_accuracy))
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

Mean CV Accuracy: 0.8159460258051808
Accuracy: 0.8491620111731844
F1 Score: 0.8187919463087249
Precision: 0.8133333333333334
Recall: 0.8243243243243243
